<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/pk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


int convertValue(int input, int defaultValue) {
    int midpoint = MIN + (MAX - MIN) / 2;
    int rangeScale = MAX - MIN;
    int newValue;

    if (input >= MIN && input <= MAX) {
        if (input == MID) {
            newValue = defaultValue;
        } else if (input < MID) {
            newValue = defaultValue * ((float)input / midpoint);
        } else {
            newValue = defaultValue + ((float)(input - MID) / midpoint) * (rangeScale - defaultValue);
        }
    }
    return newValue;
}

In [ ]:
#include <iostream>
#include <opencv2/opencv.hpp>

using namespace cv;
using namespace std;

// 彩度の計算
double calculateSaturation(const Mat &image) {
    // assertのチェックサイズとタイプ`HSV`
    Mat hsvImage;
    cvtColor(image, hsvImage, COLOR_BGR2HSV);

    Scalar mean, stddev;
    meanStdDev(hsvImage, mean, stddev);

    return (stddev[1] / 255.0) * 100;
}

// 明るさの計算
double calculateBrightness(const Mat &image) {
    Scalar meanValue = mean(image);
    double brightness = (meanValue[0] + meanValue[1] + meanValue[2]) / (3.0 * 255.0) * 100;

    return brightness;
}

// コントラストの計算
double calculateContrast(const Mat &image) {
    Mat grayImage;
    cvtColor(image, grayImage, COLOR_BGR2GRAY);

    Scalar mean, stddev;
    meanStdDev(grayImage, mean, stddev);

    return (stddev[0] / 255.0) * 100;
}

// テクスチャー強度の計算
double calculateTextureStrength(const Mat &image) {
    Mat grayImage, laplacian;
    cvtColor(image, grayImage, COLOR_BGR2GRAY);

    Laplacian(grayImage, laplacian, CV_32F, 3);

    Scalar mean, stddev;
    meanStdDev(laplacian, mean, stddev);

    return (stddev[0] / (pow(2, CV_32F) - 1)) * 100;
}

int main() {
    string imagePath = "path/to/your/image.jpg";
    Mat image = imread(imagePath);

    if (!image.data) {
        cout << "画像が読み込めませんでした。" << endl;
        return -1;
    }

    double brightness = calculateBrightness(image);
    double contrast = calculateContrast(image);
    double saturation = calculateSaturation(image);
    double textureStrength = calculateTextureStrength(image);

    cout << "明るさ: " << brightness << endl;
    cout << "コントラスト: " << contrast << endl;
    cout << "彩度: " << saturation << endl;
    cout << "テクスチャー強度: " << textureStrength << endl;

    return 0;
}


45.6

In [ ]:
#include <iostream>
#include <opencv2/opencv.hpp>

double convertRange(double input, double inMin, double inMax, double outMin, double outMax) {
    return ((input - inMin) * (outMax - outMin)) / (inMax - inMin) + outMin;
}

int main() {
    // 画像を読み込む
    cv::Mat image = cv::imread("path/to/image.jpg");

    // グレースケールに変換
    cv::Mat gray;
    cv::cvtColor(image, gray, cv::COLOR_BGR2GRAY);

    // 明るさとコントラスト
    double minVal, maxVal;
    cv::minMaxLoc(gray, &minVal, &maxVal);
    double brightness_range = maxVal - minVal;
    double brightness_normalized = convertRange(brightness_range, 0, 255, 0, 100);

    // 彩度
    cv::Mat hsv;
    cv::cvtColor(image, hsv, cv::COLOR_BGR2HSV);
    std::vector<cv::Mat> channels(3);
    cv::split(hsv, channels);
    cv::Mat saturation = channels[1];
    double saturation_average = cv::mean(saturation)[0];
    double saturation_normalized = convertRange(saturation_average, 0, 255, 0, 100);

    // 結果を表示
    std::cout << "Brightness: " << brightness_normalized << std::endl;
    std::cout << "Contrast: " << brightness_range << std::endl;
    std::cout << "Saturation: " << saturation_normalized << std::endl;

    return 0;
}


In [ ]:
#include <iostream>
#include <opencv2/core.hpp>
#include <opencv2/highgui.hpp>
#include <opencv2/imgproc.hpp>

int main(int argc, char** argv)
{
    // 画像の読み込み
    cv::Mat inputImage = cv::imread("input_image_path");

    if (inputImage.empty())
    {
        std::cout << "画像が読み込めませんでした。" << std::endl;
        return -1;
    }

    int textureStrength; // テキスチャ強度
    std::cout << "テキスチャ強度(0-100)を入力してください: ";
    std::cin >> textureStrength;

    // 入力チェック
    if (textureStrength < 0 || textureStrength > 100)
    {
        std::cout << "無効な入力です。0から100の間で入力してください。" << std::endl;
        return -1;
    }

    cv::Mat filteredImage;
    inputImage.copyTo(filteredImage);

    // 効果を調整するフィルタを適用
    double alpha = textureStrength / 100.0;

    // フィルタ1: ガウシアンブラー
    cv::GaussianBlur(filteredImage, filteredImage, cv::Size(3, 3), alpha * 2, alpha * 2);
    // フィルタ2: ラプラシアンエッジ検出
    cv::Laplacian(filteredImage, filteredImage, CV_16S, 3, 1, 0, cv::BORDER_DEFAULT);
    // フィルタ3: Bilateralフィルタ
    cv::Mat filteredImage2;
    cv::bilateralFilter(filteredImage, filteredImage2, 9, alpha * 75, alpha * 75);

    // 画像を表示
    cv::namedWindow("Input Image", cv::WINDOW_AUTOSIZE);
    cv::imshow("Input Image", inputImage);
    cv::namedWindow("Filtered Image", cv::WINDOW_AUTOSIZE);
    cv::imshow("Filtered Image", filteredImage2);

    cv::waitKey(0);
    return 0;
}

In [ ]:

def convert_brightness(user_input):
    # 사용자 입력 값이 0에서 100 사이인지 확인
    if 0 <= user_input <= 100:
        # 50일 경우 명도 값은 정확히 1.0입니다.
        if user_input == 50:
            brightness_value = 1.0
        # 10에서 49까지의 범위입니다.
        elif 10 <= user_input < 50:
            brightness_value = 1.0 - ((50 - user_input) / 40)
        # 51에서 100까지의 범위입니다.
        elif 50 < user_input <= 100:
            brightness_value = 1.0 + ((user_input - 50) / 25)
        # 0에서 9까지의 범위입니다.
        else:
            brightness_value = 0.0
    else:
        print("입력 값이 범위를 벗어났습니다.")
        return None

    return brightness_value


# 사용자 입력 값 받기
user_input = int(input("0에서 100 사이의 값을 입력하세요: "))

# 명도 조절 값 계산
brightness_value = convert_brightness(user_input)

if brightness_value is not None:
    print(f"명도 조절 값: {brightness_value}")

In [ ]:

#include <iostream>
using namespace std;

float convert_brightness(int user_input) {
    float brightness_value;

    // 사용자 입력 값이 0에서 100 사이인지 확인
    if (0 <= user_input && user_input <= 100) {
        // 50일 경우 명도 값은 정확히 1.0입니다.
        if (user_input == 50) {
            brightness_value = 1.0;
        }
        // 10에서 49까지의 범위입니다.
        else if (10 <= user_input && user_input < 50) {
            brightness_value = 1.0 - ((50 - user_input) / 40.0);
        }
        // 51에서 100까지의 범위입니다.
        else if (50 < user_input && user_input <= 100) {
            brightness_value = 1.0 + ((user_input - 50) / 25.0);
        }
        // 0에서 9까지의 범위입니다.
        else {
            brightness_value = 0.0;
        }
    }
    else {
        cout << "입력 값이 범위를 벗어났습니다." << endl;
        return -1;
    }

    return brightness_value;
}

int main() {
    int user_input;
    float brightness_value;

    cout << "0에서 100 사이의 값을 입력하세요: ";
    cin >> user_input;

    // 명도 조절 값 계산
    brightness_value = convert_brightness(user_input);

    if (brightness_value >= 0) {
        cout << "명도 조절 값: " << brightness_value << endl;
    }

    return 0;
}

In [18]:

def convert_brightness(user_input):
    # 사용자 입력 값이 0에서 100 사이인지 확인
    if 0 <= user_input <= 100:
        # 50일 경우 명도 값은 정확히 1.0입니다.
        if user_input == 50:
            brightness_value = 1.0
        # 10에서 49까지의 범위입니다.
        elif 10 <= user_input < 50:
            brightness_value = 1.0 - ((50 - user_input) / 40)
        # 51에서 100까지의 범위입니다.
        elif 50 < user_input <= 100:
            brightness_value = 1.0 + ((user_input - 50) / 25)
        # 0에서 9까지의 범위입니다.
        else:
            brightness_value = 0.0
    else:
        print("입력 값이 범위를 벗어났습니다.")
        return None

    return brightness_value


# 사용자 입력 값 받기
user_input = int(input("0에서 100 사이의 값을 입력하세요: "))

# 명도 조절 값 계산
brightness_value = convert_brightness(user_input)

if brightness_value is not None:
    print(f"명도 조절 값: {brightness_value}")

0에서 100 사이의 값을 입력하세요: 9
명도 조절 값: 0.0
